In [1]:
# Liturgický kalendár: 365 dní ako "one page" zobrazenie.
# poznámka: po vygenerovaní html súboru je potrebné ešte manuálne vložiť javascript a štýlovanie.

In [2]:
import pandas as pd
from bs4 import BeautifulSoup, Comment
import pickle

In [3]:
# Load the dictionary from the pickle file
with open('dict_of_strings.pickle', 'rb') as file:
    dict_of_strings = pickle.load(file)

In [4]:
mustra="""
<!DOCTYPE html>
<html lang="sk">
<head>
    <meta charset="UTF-8">
    <title>Liturgický kalendár 2023, skúšobná verzia</title>
    <script></script>
    <style>
    /* pridávam moje, minimal style:*/
td {
  padding: 1rem;
  /* white-space: nowrap; */
}

.lcDMY {
  white-space: nowrap;
}

    table {
        /* width: 75%; 
        margin-left: auto; 
        margin-right: auto;
        font-family: Arial, Helvetica, sans-serif; 
        line-height: 1.5; */
        border: 1px solid gray; 
        border-collapse: collapse; 
        /* padding: 1rem;     */
    }
    td {
        /* white-space: pre-wrap; */
        border: 1px solid gray;
        border-collapse: collapse;
        /* padding: 1rem;         */
    }
    th {
        /* font-size: 1.1rem; */
        border: 1px solid gray;
        border-collapse: collapse;
        /* padding: 1rem;           */
    }
    tbody {
        border-style: ridge;
        border-width: 5px;
        border-color: coral
    }
    tr td:first-child {
        text-align: center;
    }    
    </style>
</head>
<body id="lc">
    <script>init();</script>
</body>
<script>
    naDnes();
</script>
</html>
"""

In [5]:
soup1 = BeautifulSoup(mustra,'lxml')

my_table = soup1.new_tag('table') 
soup1.body.append(my_table)

In [6]:
for date_string, file_content in dict_of_strings.items():

    # týždne sú oddelené hrubšími čiarami, jednotlivé týždne sú zabalené v <tbody> elementoch
    current_date = pd.to_datetime(date_string)
    if (date_string == '2023-01-01') or current_date.day_name('sk_SK')=='Pondelok':
        tbody_here = soup1.new_tag('tbody') # create new tbody here
        soup1.table.append(tbody_here)

    new_table_row = soup1.new_tag('tr')
    tbody_here.append(new_table_row)

    ### soup2
    soup2 = BeautifulSoup(file_content[56:],'lxml') # [56:] , čiže načítavam až od druhého riadku. lebo súbory začínajú ako XML, čo vyhadzovalo zbytočný warning zo strany Parsera.
    
    elems = soup2.find_all('div', class_='lcHEADinfo') # mažeme súradnice
    for elem in elems: #  v príslušný deň ich môže byť viac
        elem.extract()        
    
    elems = soup2.find_all('div', class_='lcDENalt') # tu riešime štýlovanie a vigíliu.
    for elem in elems:
        if 'psv' in elem['class']: # rušíme štýlovanie 
            elem['class'].remove('psv') # psv ... prikázaný sviatok, rušíme štýlovanie (červený okraj)

        h3 = elem.find('h3')                       
        if h3:
            if h3.get_text().startswith('Vigília'): # ak je to vigília, mažeme.
                elem.extract()  

    # h1
    # mažeme link, aj nadbytočný <span> :
    soup2.find('a', class_='lcMY').unwrap() # alebo: soup2.h1.a.unwrap() #  lcMY
    soup2.find('span', class_='lcMD').unwrap() # lcMD
    span_lcDMY = soup2.find('span', class_='lcDMY') # lcDMY
    span_lcDMY.smooth()
    span_lcDMY.string = span_lcDMY.string[:-5] # mažeme rok 2023 z textu 
    # toto nešlo: span_lcDMY.string = span_lcDMY.string.removesuffix(' 2023') # 

    soup2.find('span', class_='lcWD').append(soup2.new_tag('br')) # <br> 
    soup2.find('span', class_='lcDMY').append(soup2.new_tag('br')) # <br>
    for elem in soup2.find_all('h1'): # nechceme nadpisy, meníme (renaming) na div
        elem.name = 'div'

    # h2        
    for elem in soup2.find_all('h2'): # nechceme nadpisy, meníme (renaming) na div
        elem.name = 'div'

    # td1 bude obsahovať: datum (lcDMY) a deň v týždni (lcWD)
    td1 = soup1.new_tag('td', id=date_string) # id='2023-01-01' napríklad. využije sa pri naDnes navigácii
    td1.append( soup2.find('span', class_='lcDMY') ) # appending is also moving...   
    td1.append( soup2.find('span', class_='lcWD') ) # appending is also moving...
    new_table_row.append(td1) # <tr><td>    

    # td2 bude obsahovať: meniny (lcND) a zvyšok main soup.
    td2 = soup1.new_tag('td')
    td2.append( soup2.find('span', class_='lcND') ) # appending is also moving...
    for elem in soup2.find_all('div', class_='lcHEAD'):
        td2.append(elem)
        
    new_table_row.append(td2) # <tr><td>    

In [7]:
# final cleaning (size reducing)

# removing html comments <!-- -->
comments = soup1.find_all(string = lambda x: isinstance(x, Comment))
for comment in comments:
    comment.extract()

# soup1.smooth() # optionally

with open('onepage.html', mode='w', encoding='utf-8') as f_out:
    f_out.write(str(soup1))

In [ ]:
# done.